# CouchDB CRUD operace prostřednictvím GraphQL

## Problémy v projektu

  ### Nefunkční soubor run.bat

Při spuštění souboru run.bat nastala chyba v prostředí Dockeru, konkrétně v oblasti import module 'couchdb', ale při lokálním spuštění proběhlo vše 
bez problémů. Domnívali jsme se, že je zřejmě chyba v requirements.

<b>Řešení:</b>

Z důvodu prvotního rozhodování, kterou z knihoven spolupracující s couchDB databází použijeme jsme v requirements měli vloženo více knihoven.
Konkrétně jsme se v requirements snažili nainstalovat dvě knihovny a to knihovnu CouchBase a CouchDB, které spolu však nespolupracují. Odstraněním knihovny CouchBase, kterou jsme se rozhodli v projektu nepoužít, byl problém vyřešen.
Nadále v projektu používáme knihovnu couchdb. Celý projekt by šel také vyřešit bez knihoven pouze pomocí HTTP požadavků.


### Chyba načítaní schémat v GQL

Při načítání schémat v GQL nastala neznámá chyba -> GRAPHQL vrací při většině chyb pouze "NonTypeError", kde nám tahle nic neříkající nápověda moc nepomůže.


<b>Řešení:</b>

V průběhu tvorby projektu a získávání nových znalostí a zkušeností s GQL jsme začali využívat vlastního debugování po řádcích, pomocí funkce print. Díky tomuhle banálnímu řešení jsme nalezli na jakém řádku nám program skončil a vznikla chyba.

V první fázi pojektu byla "NonTypeError" chyba nejčastěji způsobena chybným parsováním vrácených dat do očekávaného návratového GQL modelu. Vyřešeno vytvořením nových schémat, modelů, queries a resloverů.

V aktuální a finální verzi projektu se nemusíme zabývat parsováním dokumnetů pomocí použití funkce db.get() a správné definice resolverů.


### UpdateUser chyba

Pokud zavoláme CRUD operaci update user. Může nastat chyba při nenalezení požadovaného usera na update.

<b>Řešení:</b>

Pokud nenalezneme ID usera, tak máme více možností. Buď usera vytvoříme a nebo vrátíme chybovou hlášku, že user neexistuje a proto nemůže být updatován.

### CRUD delete 

Při použití funkce db.delete(document) se dokument z databáze odstraní pouze částečně.

Pokud se podíváme na velikost databáze, tak ta se po zavolání delete funkce nezmění i když by měla.

<br/>
<b>Řešení:</b>

Funkce delete prvek odstraní, ale ponechá jeho historii v databázi, v případě potřeby se můžeme podívat, které dokumenty byli odstraněny.

Pro kompletní delte z databáze bez ponechání historie je potřeba použít funkci db.purge(document).

### Změna složité funkce find_first()

Při prvotní tvorbě projektu jsme si vytvořili funkci, která měla za úkol najít první prvek z databáze podle zadaného _id dokumentu.

V průběhu tvorby projektu se nám nabaloval problém s vyhledáním ´dalšího prvku v prvku... např. *User*, který má v sobě *groups* a ty v sobě mají *members* proto jsme začali větvit naši funkci, kde se kontrolovali podmínky zda se v daném prvku neobjevuje prvek, který by obsahoval další prvky. Tím se nám funkce roztáhla na více jak 200 řádků a začínala být nepřehledná.

<br/>
<b>Řešení:</b>

Po odprezentování Beta verze jsme zjistili, že máme chybu v resolverech. Ty by měli být mezi sebou propojeny a pokud očekáváme pod jednou položkou jiný prvek, tak právě definovány v resolveru.

Při předělávání resolverů a celkově struktury našeho projektu, aby zapadl do měřícího mechanismu jsme začali využívat funkci db.get(), která nám vrátí požadovaný prvek z databáze doplněný o  < Dokument ... { obsah vyhledávaného prvku } >, kterého jsme se potřebovali zbavit. Po náhodném vyzkoušení funkce copy(), která se používá při kopírování dictionaries, jsme zjistili, že funkce copy() zajistí zbavení se právě námi nepožadovaného doplňku < Document >.

Proto nadále v projektu můžeme používat vestavěnou funkci db.get() na místo námi složité definované find_first.

### Relations mezi tabulkami

CouchDB je dokumnetová databáze. Její výhodou je, že nemá striktně definováno co do databáze můžete uložit, tedy každý prvek může mít jinou strukturu.

Díky téhle vlastnosti jsme do každého dokumentu doplnili pole "type" ve kterém jsme určili jakého typu je daný dokument. Téhle vlastnosti využíváme při vyhledávání jen určitého typu dokumnetů.

Dále je v dokumentové databázi horší definovat relace mezi dokumenty.
<br/>

<b>Řešení:</b>

Do dokumentu, kde byli potřebné relace jsme vložili prvek, který obsahoval [ { groupID, groupRoleID } ] Při vyhledávání prvku jsme se podívali do tohohle řádku a zjistili relace, které jsme dále podle id vyhledali v databázi.
<br/>

<b>Řešení po Beta -> Finální aplikace</b>

Po překopání projektu jsme zvolili úplně jiný způsob, abychom splňovali požadované funkcionality na měření.

Do databáze jsme začali ukládat speciální dokumenty, které určovali relace. V každém "relace" dokumentu se nachází { "_idRelace", "users_id", "groups_id", "roleType_id", "type":"relation" } v takové ralaci vyčteme jaký uživatel má jakou roli v jaké skupině.

Typy rolí a typy skupin jsou určeny napevno stanovenými hodnotami podle požadavků na projekt.
Ale není problém si vytvořit vlastní roli nebo vlastní typ skupiny.


### Create Relation

Při vytváření nových relací. Tedy přidávání usera do skupiny a přiřazování mu role. Potřebujeme vytvořit podmínky pro kontrolu, zda už stejná relace (role usera ve skupině) neexistuje. V případě, že existuje vrátíme prvek téhlé relace. Každý user může být ve více skupinách a může mít více rolí.